# **Applying Base Classification Model**

In this notebook, we use the dataset created in the first notebook to apply base classification models and assess their accuracies.

For each model, we created a pipeline that makes it easier to apply the vectorization, and tokenization process

## **Important Installations**

In [ ]:
!pip install --upgrade gensim
!pip install xgboost
!pip install snowballstemmer
!pip install sentencepiece
!pip install lime

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [ ]:
pip install lime

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Tokenization
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py    

## **Uploading Important libarires**

In [ ]:
import pandas as pd
import numpy as np
import os

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


import tokenization


import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams



from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam


import logging
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

## Retrieve dataset from first analysis

In [ ]:
df_main = pd.read_csv('/content/drive/MyDrive/WSU/Winter 2021/CSC 7810/Final Project/df_main_res.csv')
df_main['review'] = df_main['review'].values.astype('U')
df_main['label'] = df_main['target'].apply(lambda x: "Positive" if x==1 else "Negative")
neg, pos = np.bincount(df_main['target'])

### Oversample the negative class

In [ ]:

"""
features = df_main['review']
labels = df_main['target']

pos_features = df_main['review'].loc[(df_main['target'] == 1)] 
neg_features = df_main['review'].loc[(df_main['target'] == 0)] 

pos_labels = df_main['target'].loc[(df_main['target'] == 1)] 
neg_labels = df_main['target'].loc[(df_main['target'] == 0)] 
"""
"""
ids = np.arange(len(neg_features))
choices = np.random.choice(ids, len(pos_features))

res_neg_features =  neg_features.reindex(choices)
res_neg_labels = neg_labels.reindex(choices)

res_neg_features.shape

resampled_features = np.concatenate([res_neg_features, pos_features], axis=0)
resampled_labels = np.concatenate([res_neg_labels, pos_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape


"""

'\nids = np.arange(len(neg_features))\nchoices = np.random.choice(ids, len(pos_features))\n\nres_neg_features =  neg_features.reindex(choices)\nres_neg_labels = neg_labels.reindex(choices)\n\nres_neg_features.shape\n\nresampled_features = np.concatenate([res_neg_features, pos_features], axis=0)\nresampled_labels = np.concatenate([res_neg_labels, pos_labels], axis=0)\n\norder = np.arange(len(resampled_labels))\nnp.random.shuffle(order)\nresampled_features = resampled_features[order]\nresampled_labels = resampled_labels[order]\n\nresampled_features.shape\n\n\n'

In [ ]:
"""
import tensorflow as tf
BUFFER_SIZE = 100000
BATCH_SIZE = 2048

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)
"""

In [ ]:
"""
for features, label in neg_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())
"""

Features:
 b'used efudix day got bearable symptom rash redness sore skin took month redness completely main long lasting side effect six month still permanently greasy greasy skin nose cheek applied tip make sure prone rosacea using know prone efudix triggered adult rosacea never knew rosacea like acne come go place used efudix embarrassing like going public flare ups would recommend treatment might small 2cm spot treatment body face beware'

Label:  0


In [ ]:
"""
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)
"""

In [ ]:
"""
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

"""

0.50146484375


### Split data into training and testing sets

In [ ]:
X = df_main['review']
y = df_main['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
#v = TfidfVectorizer(decode_error='replace', encoding='utf-8')
#x = v.fit_transform(df_main['review'].apply(lambda x: np.str_(x)))

## **Models**

We applied 5 models:


1.   Naive Bayes: Nultinomial
2.   Support Vector Machine: Stochastic Gradient Descent
3.   Log Regression
4.   Gradient Boosting Classifier
5.   XGBoost



### First Pipeline: Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('ROC AUC %s' % roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy 0.839433236094428
ROC AUC 0.8394659806770043
              precision    recall  f1-score   support

           0       0.83      0.85      0.84     20325
           1       0.85      0.83      0.84     20468

    accuracy                           0.84     40793
   macro avg       0.84      0.84      0.84     40793
weighted avg       0.84      0.84      0.84     40793



### Second Pipeline: Support Vector Machine

In [ ]:
%%time
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)



y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('ROC AUC %s' % roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


accuracy 0.8324222293040473
ROC AUC 0.8323588123999278
              precision    recall  f1-score   support

           0       0.84      0.81      0.83     20325
           1       0.82      0.85      0.84     20468

    accuracy                           0.83     40793
   macro avg       0.83      0.83      0.83     40793
weighted avg       0.83      0.83      0.83     40793

CPU times: user 5.63 s, sys: 3.95 s, total: 9.58 s
Wall time: 5 s


### Third Pipeline: Logistic Regression

In [ ]:

%%time

from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('ROC AUC %s' % roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy 0.8926776652857108
ROC AUC 0.8927000644933164
              precision    recall  f1-score   support

           0       0.89      0.90      0.89     20325
           1       0.90      0.89      0.89     20468

    accuracy                           0.89     40793
   macro avg       0.89      0.89      0.89     40793
weighted avg       0.89      0.89      0.89     40793

CPU times: user 25 s, sys: 2min 46s, total: 3min 11s
Wall time: 9.44 s


### Fourth Pipeline: Gradient Boosting Classifier

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier

GBC = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier(random_state=5)),
               ])
GBC.fit(X_train, y_train)

y_pred = GBC.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('ROC AUC %s' % roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy 0.7771676513127252
ROC AUC 0.7773324862425874
              precision    recall  f1-score   support

           0       0.75      0.82      0.79     20325
           1       0.81      0.73      0.77     20468

    accuracy                           0.78     40793
   macro avg       0.78      0.78      0.78     40793
weighted avg       0.78      0.78      0.78     40793

CPU times: user 2min 15s, sys: 30.9 ms, total: 2min 15s
Wall time: 2min 15s


### Fifth Pipeline: XGBoost

In [ ]:
%%time
from xgboost import XGBClassifier

from sklearn.ensemble import GradientBoostingClassifier

XGBO = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', XGBClassifier(random_state=5)),
               ])
XGBO.fit(X_train, y_train)

y_pred = XGBO.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('ROC AUC %s' % roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy 0.7760890348834358
ROC AUC 0.7762803245386372
              precision    recall  f1-score   support

           0       0.75      0.83      0.79     20325
           1       0.81      0.72      0.76     20468

    accuracy                           0.78     40793
   macro avg       0.78      0.78      0.78     40793
weighted avg       0.78      0.78      0.78     40793

CPU times: user 50.5 s, sys: 76.3 ms, total: 50.6 s
Wall time: 50.4 s


#**Experiment with LIME**

Actual results in separate notebook

In [ ]:
# LIME SECTION
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [ ]:
from __future__ import print_function
predict_fn_nb = lambda x: nb.predict_proba(X_test).astype(float)
predict_fn_sgd = lambda x: sgd.predict_proba(X_test).astype(float)
predict_fn_logreg = lambda x: logreg.predict_proba(X_test).astype(float)
predict_fn_gbc = lambda x: gbc.predict_proba(X_test).astype(float)
predict_fn_xgbo = lambda x: XGBO.predict_proba(X_test).astype(float)

In [ ]:
# Line-up the feature names
"""
feature_names_cat = list(train_cat_features_ver2)
feature_names_float = list(train_float_features)
feature_names_int = list(train_int_features)

feature_names = sum([feature_names_cat, feature_names_float, feature_names_int], [])
print(feature_names)
"""


NameError: ignored

In [ ]:
"""
# Create the LIME Explainer
idx = 18433
class_names = list(df_main.label.unique())
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(X_test[idx], nb.predict_proba, num_features=100, labels=[0, 1])
"""

In [ ]:
"""
print('Document id: %d' % idx)
print('Predicted class =', class_names[nb.predict(training_feature[idx]).reshape(1,-1)[0,0]])
print('True class: %s' % class_names[y_test[idx]])
"""

In [ ]:
"""
print ('Explanation for class %s' % class_names[1])
print ('\n'.join(map(str, exp.as_list(label=1))))
"""

In [ ]:
#exp.show_in_notebook(text=y_test[idx], labels=(0,))